In [1]:
from datetime import datetime
import os
import gzip
import pandas as pd
import numpy as np

today = datetime.today().strftime('%Y-%m-%d')

os.chdir(r'D:\TD\FMF\Strategies')
filename = 'StrategyTestingResults v2.csv.gz'

In [2]:
###Load the data

with gzip.open(filename, 'rt') as file:
    data = pd.read_csv(file, parse_dates=['entry_time', 'exit_time'], index_col=None, dayfirst=True)
    data['entry_time'] = pd.to_datetime(data['entry_time'])#, dayfirst=True)
    data['exit_time'] = pd.to_datetime(data['exit_time'])#, dayfirst=True)
    data = data.sort_values(['entry_time', 'exit_time'], ascending = (True, True))
    data.dropna(axis=0, how='all', inplace=True)
    data = data.drop_duplicates(subset=['ticker', 'timeframe', 'strategy', 'entry_price', 'entry_time', 
                'exit_price', 'exit_time'], keep='last', inplace=False)#, ignore_index=True)
    #data['profit%'] = np.where(data['entry_signal'] == 'BUY', (((((10000 * 0.992) / data['entry_price']) * data['exit_price']) - (10000 * 0.996)) / 100), (((10000 * 0.996) - (((10000 * 0.992) / data['entry_price']) * data['exit_price'])) / 100)) #I am using 0.992 since the second trasaction uses the amount after the first commission and a second .004 commission is added.
    data['profit%'] = np.where(data['entry_signal'] == 'BUY', ((data['exit_price'] - data['entry_price']) * 10000 - 8) / 10000, ((data['entry_price'] - data['exit_price']) * 10000 - 8) / 10000) #made it simpler here: Binance is charging me 8 USD for each 10k transaction

#Computing the cummulative profit (includes the trade direction: BUY or SELL). This basically adds up the profit after each trade.
data['CumulativeProfit%'] = data.groupby(['ticker', 'strategy', 'timeframe', 'entry_signal'])['profit%'].cumsum() / 100
data['CumulativeProfit%'] = data['CumulativeProfit%'].apply(lambda x: f'{x:.2%}')

#Computing the cumulative profit by strategy without the direction of the trade.
data['CumulativeProfitStrategy%'] = data.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].cumsum() / 100
data['CumulativeProfitStrategy%'] = data['CumulativeProfitStrategy%'].apply(lambda x: f'{x:.2%}')

#Computing how long the trade was in the market
data['TimeInMarket'] = (data['exit_time'] - data['entry_time']).dt.total_seconds() / 3600.0

#Getting the day of the week when the trade was placed
data['WeekDayEntry'] = data['entry_time'].dt.dayofweek

#Getting the hour when the trade was placed
data['HourOfEntry'] = data['entry_time'].dt.hour

#Checking the data
data

,ticker,timeframe,strategy,entry_price,entry_time,entry_signal,exit_price,exit_time,exit_signal,runup%,drawdown%,profit%,CumulativeProfit%,CumulativeProfitStrategy%,TimeInMarket,WeekDayEntry,HourOfEntry
0,BINANCE:ETHUSDT.P,1m,[CP] ChannelBreakOutStrategy,1579.80,2023-09-25 00:00:00,SELL,1578.12,2023-09-25 00:13:00,BUY,0.30,0.06,1.6792,1.68%,1.68%,0.216667,0,0
1,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,208.20,2023-09-25 00:00:00,SELL,207.81,2023-09-25 00:14:00,BUY,0.36,0.06,0.3892,0.39%,0.39%,0.233333,0,0
2,BINANCE:BTCUSDT.P,1m,[CP] ChannelBreakOutStrategy,26213.40,2023-09-25 00:00:00,SELL,26180.10,2023-09-25 00:15:00,BUY,0.30,0.08,33.2992,33.30%,33.30%,0.250000,0,0
3,BINANCE:ETHUSDT.P,1m,[CP] ChannelBreakOutStrategy,1578.12,2023-09-25 00:13:00,BUY,1576.86,2023-09-25 00:18:00,SELL,0.04,0.08,-1.2608,-1.26%,0.42%,0.083333,0,0
4,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,207.81,2023-09-25 00:14:00,BUY,207.53,2023-09-25 00:21:00,SELL,0.04,0.15,-0.2808,-0.28%,0.11%,0.116667,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16331,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,205.47,2023-10-12 14:52:00,SELL,205.57,2023-10-12 14:56:00,BUY,0.07,0.05,-0.1008,31.55%,60.48%,0.066667,3,14
16332,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,205.57,2023-10-12 14:56:00,BUY,205.50,2023-10-12 15:02:00,SELL,0.04,0.04,-0.0708,28.85%,60.41%,0.100000,3,14
16333,BINANCE:BNBUSDT.P,1m,[CP] Momentum Strategy,205.59,2023-10-12 14:57:00,BUY,205.59,2023-10-12 15:00:00,SELL,0.03,0.02,-0.0008,12.06%,26.12%,0.050000,3,14
16334,BINANCE:BNBUSDT.P,1m,[CP] Momentum Strategy,205.59,2023-10-12 15:00:00,SELL,205.55,2023-10-12 15:05:00,BUY,0.13,0.00,0.0392,14.10%,26.16%,0.083333,3,15


In [3]:
#Cleaning the data so that the COMMENTS ARE ALWAYS BUY/SELL, CLOSE BUY/CLOSE SELL

data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('Entry Long', 'BUY'))
data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('Entry Short', 'SELL'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('Exit Long', 'CLOSE BUY'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('Exit Short', 'CLOSE SELL'))
data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('MomLE', 'BUY'))
data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('MomSE', 'SELL'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('MomLE', 'SELL'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('MomSE', 'BUY'))


In [4]:
#####RESAVE DATA TO CSV
os.chdir(r'D:\TD\FMF\Strategies')
file = 'StrategyTestingResults v2 for Analysis.csv.gz'
data.to_csv(file, index=False, compression='gzip')

#############################################<b>OverallStrategy Processing</b>#####################################

In [5]:
######Summing up the profit for each tuple 'ticker, strategy and timeframe' in order to compare them. 
totalprofit2 = pd.DataFrame()
totalprofit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].sum().reset_index()
totalprofit2.columns = ['ticker', 'strategy', 'timeframe', 'TotalProfit%']

#Getting the average profit per trade for each tuple of ticker, strategy, timeframe
avgprofit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].mean().reset_index()#.apply(lambda x: f'{x:.2f}').reset_index()
avgprofit2.columns = ['ticker', 'strategy', 'timeframe', 'AvgProfitPerTrade%']
prof12 = avgprofit2.merge(totalprofit2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])

#Computing the lowest value of the cumulative profit (after each trade)
mincumprofitp2 = data.groupby(['ticker', 'strategy', 'timeframe'])['CumulativeProfitStrategy%'].min().reset_index()
mincumprofitp2.columns = ['ticker', 'strategy', 'timeframe', 'MinCumulative%']
prof22 = mincumprofitp2.merge(prof12, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])

#Computing the maximum value of the cumulative profit (after each trade)
maxcumprofitp2 = data.groupby(['ticker', 'strategy', 'timeframe'])['CumulativeProfitStrategy%'].max().reset_index()
maxcumprofitp2.columns = ['ticker', 'strategy', 'timeframe', 'MaxCumulative%']
profit2 = maxcumprofitp2.merge(prof22, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
profit2 #check that it works

,ticker,strategy,timeframe,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%
0,BINANCE:BNBUSDT.P,Strategy v1,1m,8.13%,1.77%,5.685867,17.0576
1,BINANCE:BNBUSDT.P,Strategy v10,1m,1.97%,-0.53%,0.271200,1.3560
2,BINANCE:BNBUSDT.P,Strategy v15,1m,8.24%,-2.85%,3.120629,21.8444
3,BINANCE:BNBUSDT.P,Strategy v16,1m,0.53%,0.53%,0.529200,0.5292
4,BINANCE:BNBUSDT.P,Strategy v19,1m,9.77%,-0.30%,-0.424510,-26.3196
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,9.92%,-0.14%,0.201481,34.4532
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,9.54%,-0.62%,0.213129,5.9676
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,9.98%,-0.09%,0.021982,60.2088
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,2.66%,-0.00%,-0.007739,-3.3124
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,9.93%,-0.17%,0.014756,26.1616


In [6]:
################################################################################################################################
#Determine the interval in which the Strategy has been active for the ticker and timeframe and get the result in days
MinDate2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_time'].min().reset_index()
MaxDate2 = data.groupby(['ticker', 'strategy', 'timeframe'])['exit_time'].max().reset_index()
m2 = MaxDate2.merge(profit2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2 = MinDate2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2['TimeInDays'] = (m2.exit_time - m2.entry_time).dt.total_seconds() / 86400
m2

,ticker,strategy,timeframe,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:BNBUSDT.P,Strategy v1,1m,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528
1,BINANCE:BNBUSDT.P,Strategy v10,1m,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111
2,BINANCE:BNBUSDT.P,Strategy v15,1m,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583
3,BINANCE:BNBUSDT.P,Strategy v16,1m,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972
4,BINANCE:BNBUSDT.P,Strategy v19,1m,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583


In [7]:
################################################################################################################################
#Compute the number of trades the strategy has placed for the ticker and timeframe within the interval
Trades2 = pd.DataFrame()
Trades2 = data.groupby(['ticker', 'strategy', 'timeframe']).size().reset_index()
Trades2.columns = ['ticker', 'strategy', 'timeframe', 'NoOfTrades']
m2 = Trades2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528
1,BINANCE:BNBUSDT.P,Strategy v10,1m,5,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111
2,BINANCE:BNBUSDT.P,Strategy v15,1m,7,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583
3,BINANCE:BNBUSDT.P,Strategy v16,1m,1,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972
4,BINANCE:BNBUSDT.P,Strategy v19,1m,62,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,171,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,28,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,2739,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,428,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,1773,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583


In [8]:
################################################################################################################################
#Get the average drawdown of the strategy for the tuple/timeframe
drawdown2  = pd.DataFrame()
drawdown2 = data.groupby(['ticker', 'strategy', 'timeframe'])['drawdown%'].mean().apply(lambda x: f'{x:.1f}').reset_index()
m2 = drawdown2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:BNBUSDT.P,Strategy v1,1m,2.9,3,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528
1,BINANCE:BNBUSDT.P,Strategy v10,1m,0.4,5,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111
2,BINANCE:BNBUSDT.P,Strategy v15,1m,1.0,7,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583
3,BINANCE:BNBUSDT.P,Strategy v16,1m,1.5,1,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972
4,BINANCE:BNBUSDT.P,Strategy v19,1m,0.9,62,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,0.9,171,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,1.3,28,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,0.0,2739,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,0.1,428,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,0.1,1773,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583


In [9]:
################################################################################################################################
#Get the average runup of the strategy for the tuple/timeframe
runup2 = pd.DataFrame()
runup2 = data.groupby(['ticker', 'strategy', 'timeframe'])['runup%'].mean().apply(lambda x: f'{x:.1f}').reset_index()
runup2.columns = ['ticker', 'strategy', 'timeframe', 'runup%']
m2 = runup2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,runup%,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3.1,2.9,3,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528
1,BINANCE:BNBUSDT.P,Strategy v10,1m,1.4,0.4,5,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111
2,BINANCE:BNBUSDT.P,Strategy v15,1m,2.3,1.0,7,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583
3,BINANCE:BNBUSDT.P,Strategy v16,1m,3.9,1.5,1,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972
4,BINANCE:BNBUSDT.P,Strategy v19,1m,0.5,0.9,62,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,0.5,0.9,171,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,0.9,1.3,28,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,0.1,0.0,2739,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,0.3,0.1,428,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,0.1,0.1,1773,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583


In [10]:
################################################################################################################################
#Get the average time in market (in hours) of the strategy for the ticker/timeframe
avgtimeinmkt2 = pd.DataFrame()
avgtimeinmkt2 = data.groupby(['ticker', 'strategy', 'timeframe'])['TimeInMarket'].mean().reset_index()
avgtimeinmkt2.columns = ['ticker', 'strategy', 'timeframe', 'AvgTimeInMarketHours']
avgtimeinmkt2['AvgTimeInMarketHours'] = (avgtimeinmkt2['AvgTimeInMarketHours'] / 60)
m2 = avgtimeinmkt2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,AvgTimeInMarketHours,runup%,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3.738704,3.1,2.9,3,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528
1,BINANCE:BNBUSDT.P,Strategy v10,1m,0.144444,1.4,0.4,5,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111
2,BINANCE:BNBUSDT.P,Strategy v15,1m,0.985040,2.3,1.0,7,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583
3,BINANCE:BNBUSDT.P,Strategy v16,1m,2.026389,3.9,1.5,1,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972
4,BINANCE:BNBUSDT.P,Strategy v19,1m,0.123217,0.5,0.9,62,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,0.187326,0.5,0.9,171,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,0.295397,0.9,1.3,28,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,0.002575,0.1,0.0,2739,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,0.016441,0.3,0.1,428,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,0.003974,0.1,0.1,1773,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583


In [11]:
################################################################################################################################
#Get the standard deviation of the time in market (in hours) of the strategy for the tuple/timeframe
stdtimeinmkt2 = pd.DataFrame()
stdtimeinmkt2 = data.groupby(['ticker', 'strategy', 'timeframe'])['TimeInMarket'].std().reset_index()
stdtimeinmkt2.columns = ['ticker', 'strategy', 'timeframe', 'StdTimeInMarket']
stdtimeinmkt2['StdTimeInMarket'] = (stdtimeinmkt2['StdTimeInMarket'] / 60)
m2 = stdtimeinmkt2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,StdTimeInMarket,AvgTimeInMarketHours,runup%,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:BNBUSDT.P,Strategy v1,1m,1.172913,3.738704,3.1,2.9,3,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528
1,BINANCE:BNBUSDT.P,Strategy v10,1m,0.086055,0.144444,1.4,0.4,5,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111
2,BINANCE:BNBUSDT.P,Strategy v15,1m,0.515875,0.985040,2.3,1.0,7,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583
3,BINANCE:BNBUSDT.P,Strategy v16,1m,NaN,2.026389,3.9,1.5,1,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972
4,BINANCE:BNBUSDT.P,Strategy v19,1m,0.126077,0.123217,0.5,0.9,62,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,0.173116,0.187326,0.5,0.9,171,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,0.391121,0.295397,0.9,1.3,28,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,0.001906,0.002575,0.1,0.0,2739,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,0.013927,0.016441,0.3,0.1,428,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,0.003397,0.003974,0.1,0.1,1773,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583


In [12]:
################################################################################################################################
#Get the min and max price within the interval (for volatility)
minentry2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].min().reset_index()
minentry2.columns = ['ticker', 'strategy', 'timeframe', 'MinPriceEntry']
m2 = minentry2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
maxentry2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].max().reset_index()
maxentry2.columns = ['ticker', 'strategy', 'timeframe', 'MaxPriceEntry']
m2 = maxentry2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
minexit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].min().reset_index()
minexit2.columns = ['ticker', 'strategy', 'timeframe', 'MinPriceExit']
m2 = minexit2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
maxexit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].max().reset_index()
maxexit2.columns = ['ticker', 'strategy', 'timeframe', 'MaxPriceExit']
m2 = maxexit2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2['MinPrice'] = np.where(m2['MinPriceEntry'] <= m2['MinPriceExit'], m2['MinPriceEntry'], m2['MinPriceExit'])
m2['MaxPrice'] = np.where(m2['MaxPriceEntry'] >= m2['MaxPriceExit'], m2['MaxPriceEntry'], m2['MaxPriceExit'])
m2['var'] = np.where(m2['MinPrice'] <= m2['MaxPrice'], ((m2['MaxPrice'] - m2['MinPrice']) / m2['MinPrice']), 1000)
m2['var'] = m2['var'].apply(lambda x: f'{x:.1f}%')
m2

,ticker,strategy,timeframe,MaxPriceExit,MinPriceExit,MaxPriceEntry,MinPriceEntry,StdTimeInMarket,AvgTimeInMarketHours,runup%,...,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var
0,BINANCE:BNBUSDT.P,Strategy v1,1m,218.79,211.63,218.79,211.63,1.172913,3.738704,3.1,...,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528,211.63,218.79,0.0%
1,BINANCE:BNBUSDT.P,Strategy v10,1m,214.47,211.65,214.47,211.65,0.086055,0.144444,1.4,...,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111,211.65,214.47,0.0%
2,BINANCE:BNBUSDT.P,Strategy v15,1m,216.27,211.75,216.27,211.75,0.515875,0.985040,2.3,...,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583,211.75,216.27,0.0%
3,BINANCE:BNBUSDT.P,Strategy v16,1m,213.46,213.46,213.46,213.46,NaN,2.026389,3.9,...,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972,213.46,213.46,0.0%
4,BINANCE:BNBUSDT.P,Strategy v19,1m,218.59,204.60,218.59,204.60,0.126077,0.123217,0.5,...,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306,204.60,218.59,0.1%
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,218.99,204.97,218.99,204.97,0.173116,0.187326,0.5,...,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222,204.97,218.99,0.1%
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,218.52,202.87,218.52,202.87,0.391121,0.295397,0.9,...,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306,202.87,218.52,0.1%
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,221.11,203.79,221.11,203.79,0.001906,0.002575,0.1,...,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167,203.79,221.11,0.1%
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,220.19,204.99,220.19,204.99,0.013927,0.016441,0.3,...,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361,204.99,220.19,0.1%
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,221.08,203.98,221.08,203.98,0.003397,0.003974,0.1,...,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583,203.98,221.08,0.1%


In [13]:
#Compute the number of trades with positive and negative outcome 
################################################################################################################################

negativeonly2 = pd.DataFrame()
positiveonly2 = pd.DataFrame()
negativeonly2 = data[data['profit%'] <= 0]
positiveonly2 = data[data['profit%'] > 0]

In [14]:
################################################################################################################################
#Compute percentage of negative trades for benchmarking
NoNegative2 = pd.DataFrame()
NoNegative2 = negativeonly2.groupby(['ticker', 'strategy', 'timeframe']).size().reset_index()
NoNegative2.columns = ['ticker', 'strategy', 'timeframe', 'NoOfNegative']
ProfitNegative2 = negativeonly2.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].sum().reset_index()
ProfitNegative2.columns = ['ticker', 'strategy', 'timeframe', 'ProfitNegative%']
OnNegative2 = NoNegative2.merge(ProfitNegative2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2 = OnNegative2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,MinPriceEntry,StdTimeInMarket,...,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var
0,BINANCE:BNBUSDT.P,Strategy v10,1m,3.0,-3.0624,214.47,211.65,214.47,211.65,0.086055,...,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111,211.65,214.47,0.0%
1,BINANCE:BNBUSDT.P,Strategy v15,1m,2.0,-5.6616,216.27,211.75,216.27,211.75,0.515875,...,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583,211.75,216.27,0.0%
2,BINANCE:BNBUSDT.P,Strategy v19,1m,36.0,-55.3288,218.59,204.60,218.59,204.60,0.126077,...,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306,204.60,218.59,0.1%
3,BINANCE:BNBUSDT.P,Strategy v2.1,1m,58.0,-71.3464,218.99,204.97,218.99,204.97,0.173116,...,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222,204.97,218.99,0.1%
4,BINANCE:BNBUSDT.P,Strategy v9.1,1m,12.0,-29.3196,218.52,202.87,218.52,202.87,0.391121,...,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306,202.87,218.52,0.1%
5,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,1600.0,-175.7600,221.11,203.79,221.11,203.79,0.001906,...,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167,203.79,221.11,0.1%
6,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,290.0,-76.1620,220.19,204.99,220.19,204.99,0.013927,...,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361,204.99,220.19,0.1%
7,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,1136.0,-144.0188,221.08,203.98,221.08,203.98,0.003397,...,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583,203.98,221.08,0.1%
8,BINANCE:BTCUSDT.P,Strategy v1,1m,3.0,-2729.7024,27800.40,26348.70,27800.40,26348.70,0.855086,...,2023-09-27 09:57:00,2023-10-09 09:54:00,-2729.70%,-1175.40%,-613.350800,-2453.4032,11.997917,26348.70,27800.40,0.1%
9,BINANCE:BTCUSDT.P,Strategy v10,1m,3.0,-470.3024,27725.10,26081.30,27725.10,26081.30,0.110557,...,2023-09-25 11:00:00,2023-10-11 09:54:00,936.10%,-174.30%,142.632533,855.7952,15.954167,26081.30,27725.10,0.1%


In [15]:
################################################################################################################################
#Compute percentage of positive trades for benchmarking
NoPositive2 = pd.DataFrame()
NoPositive2 = positiveonly2.groupby(['ticker', 'strategy', 'timeframe']).size().reset_index()
NoPositive2.columns = ['ticker', 'strategy', 'timeframe', 'NoOfPositive']
ProfitPositive2 = positiveonly2.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].sum().reset_index()
ProfitPositive2.columns = ['ticker', 'strategy', 'timeframe', 'ProfitPositive%']
OnPositive2 = NoPositive2.merge(ProfitPositive2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2 = OnPositive2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2['NoOfPositive'] = m2['NoOfPositive'].fillna(0)
m2['ProfitPositive%'] = m2['ProfitPositive%'].fillna(0)
m2['NoOfNegative'] = m2['NoOfNegative'].fillna(0)
m2['ProfitNegative%'] = m2['ProfitNegative%'].fillna(0)
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3.0,17.0576,0.0,0.0000,218.79,211.63,218.79,...,2023-09-26 01:57:00,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528,211.63,218.79,0.0%
1,BINANCE:BNBUSDT.P,Strategy v10,1m,2.0,4.4184,3.0,-3.0624,214.47,211.65,214.47,...,2023-09-26 06:00:00,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111,211.65,214.47,0.0%
2,BINANCE:BNBUSDT.P,Strategy v15,1m,5.0,27.5060,2.0,-5.6616,216.27,211.75,216.27,...,2023-09-26 01:54:00,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583,211.75,216.27,0.0%
3,BINANCE:BNBUSDT.P,Strategy v16,1m,1.0,0.5292,0.0,0.0000,213.46,213.46,213.46,...,2023-10-01 17:56:00,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972,213.46,213.46,0.0%
4,BINANCE:BNBUSDT.P,Strategy v19,1m,26.0,29.0092,36.0,-55.3288,218.59,204.60,218.59,...,2023-09-25 01:45:00,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306,204.60,218.59,0.1%
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,113.0,105.7996,58.0,-71.3464,218.99,204.97,218.99,...,2023-09-25 06:33:00,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222,204.97,218.99,0.1%
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,16.0,35.2872,12.0,-29.3196,218.52,202.87,218.52,...,2023-09-27 06:25:00,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306,202.87,218.52,0.1%
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,1139.0,235.9688,1600.0,-175.7600,221.11,203.79,221.11,...,2023-09-25 00:00:00,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167,203.79,221.11,0.1%
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,138.0,72.8496,290.0,-76.1620,220.19,204.99,220.19,...,2023-09-25 00:22:00,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361,204.99,220.19,0.1%
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,637.0,170.1804,1136.0,-144.0188,221.08,203.98,221.08,...,2023-09-25 00:20:00,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583,203.98,221.08,0.1%


In [16]:
################################################################################################################################
#Compute ratio [-100 to 100] as division of profit made from positive trades divided by profit made from negative trades. If not negative trades, the ratio is 100
#Similar to Profit Factor in TradingView Statistics
m2['ratio'] = np.where(m2['ProfitNegative%'] != 0.00, m2['ProfitPositive%'] / (m2['ProfitNegative%'] * -1), 100)
m2['ratio'] = np.where(m2['ProfitPositive%'] == 0, -100, m2['ratio'])
m2['ratio'] = m2['ratio'].apply(lambda x: f'{x:.1f}')
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var,ratio
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3.0,17.0576,0.0,0.0000,218.79,211.63,218.79,...,2023-10-08 10:52:00,8.13%,1.77%,5.685867,17.0576,12.371528,211.63,218.79,0.0%,100.0
1,BINANCE:BNBUSDT.P,Strategy v10,1m,2.0,4.4184,3.0,-3.0624,214.47,211.65,214.47,...,2023-10-04 08:04:00,1.97%,-0.53%,0.271200,1.3560,8.086111,211.65,214.47,0.0%,1.4
2,BINANCE:BNBUSDT.P,Strategy v15,1m,5.0,27.5060,2.0,-5.6616,216.27,211.75,216.27,...,2023-10-09 16:03:00,8.24%,-2.85%,3.120629,21.8444,13.589583,211.75,216.27,0.0%,4.9
3,BINANCE:BNBUSDT.P,Strategy v16,1m,1.0,0.5292,0.0,0.0000,213.46,213.46,213.46,...,2023-10-06 19:31:00,0.53%,0.53%,0.529200,0.5292,5.065972,213.46,213.46,0.0%,100.0
4,BINANCE:BNBUSDT.P,Strategy v19,1m,26.0,29.0092,36.0,-55.3288,218.59,204.60,218.59,...,2023-10-12 10:44:00,9.77%,-0.30%,-0.424510,-26.3196,17.374306,204.60,218.59,0.1%,0.5
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,113.0,105.7996,58.0,-71.3464,218.99,204.97,218.99,...,2023-10-12 02:53:00,9.92%,-0.14%,0.201481,34.4532,16.847222,204.97,218.99,0.1%,1.5
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,16.0,35.2872,12.0,-29.3196,218.52,202.87,218.52,...,2023-10-10 22:36:00,9.54%,-0.62%,0.213129,5.9676,13.674306,202.87,218.52,0.1%,1.2
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,1139.0,235.9688,1600.0,-175.7600,221.11,203.79,221.11,...,2023-10-12 15:06:00,9.98%,-0.09%,0.021982,60.2088,17.629167,203.79,221.11,0.1%,1.3
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,138.0,72.8496,290.0,-76.1620,220.19,204.99,220.19,...,2023-10-12 14:35:00,2.66%,-0.00%,-0.007739,-3.3124,17.592361,204.99,220.19,0.1%,1.0
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,637.0,170.1804,1136.0,-144.0188,221.08,203.98,221.08,...,2023-10-12 15:05:00,9.93%,-0.17%,0.014756,26.1616,17.614583,203.98,221.08,0.1%,1.2


In [17]:
################################################################################################################################
#Ratio of negative trades out of total number of trades
m2['RatioNegative'] = np.where(m2['NoOfTrades'] > 0, m2['NoOfNegative'] / m2['NoOfTrades'], 0)
m2['RatioNegative'] = m2['RatioNegative'].apply(lambda x: f'{x:.0%}')
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var,ratio,RatioNegative
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3.0,17.0576,0.0,0.0000,218.79,211.63,218.79,...,8.13%,1.77%,5.685867,17.0576,12.371528,211.63,218.79,0.0%,100.0,0%
1,BINANCE:BNBUSDT.P,Strategy v10,1m,2.0,4.4184,3.0,-3.0624,214.47,211.65,214.47,...,1.97%,-0.53%,0.271200,1.3560,8.086111,211.65,214.47,0.0%,1.4,60%
2,BINANCE:BNBUSDT.P,Strategy v15,1m,5.0,27.5060,2.0,-5.6616,216.27,211.75,216.27,...,8.24%,-2.85%,3.120629,21.8444,13.589583,211.75,216.27,0.0%,4.9,29%
3,BINANCE:BNBUSDT.P,Strategy v16,1m,1.0,0.5292,0.0,0.0000,213.46,213.46,213.46,...,0.53%,0.53%,0.529200,0.5292,5.065972,213.46,213.46,0.0%,100.0,0%
4,BINANCE:BNBUSDT.P,Strategy v19,1m,26.0,29.0092,36.0,-55.3288,218.59,204.60,218.59,...,9.77%,-0.30%,-0.424510,-26.3196,17.374306,204.60,218.59,0.1%,0.5,58%
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,113.0,105.7996,58.0,-71.3464,218.99,204.97,218.99,...,9.92%,-0.14%,0.201481,34.4532,16.847222,204.97,218.99,0.1%,1.5,34%
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,16.0,35.2872,12.0,-29.3196,218.52,202.87,218.52,...,9.54%,-0.62%,0.213129,5.9676,13.674306,202.87,218.52,0.1%,1.2,43%
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,1139.0,235.9688,1600.0,-175.7600,221.11,203.79,221.11,...,9.98%,-0.09%,0.021982,60.2088,17.629167,203.79,221.11,0.1%,1.3,58%
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,138.0,72.8496,290.0,-76.1620,220.19,204.99,220.19,...,2.66%,-0.00%,-0.007739,-3.3124,17.592361,204.99,220.19,0.1%,1.0,68%
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,637.0,170.1804,1136.0,-144.0188,221.08,203.98,221.08,...,9.93%,-0.17%,0.014756,26.1616,17.614583,203.98,221.08,0.1%,1.2,64%


In [18]:
################################################################################################################################
#Adjustment in case of using leverage: settings are x100 for FX and x10 for crypto
m2['AdjustedProfit%'] = np.where(m2['ticker'].str.contains('FX:'), m2['TotalProfit%'] * 100, m2['TotalProfit%'])
m2['AdjustedProfit%'] = np.where(m2['ticker'].str.contains('BINANCE:') | m2['ticker'].str.contains('BITMEX:'), m2['TotalProfit%'] * 10, m2['AdjustedProfit%'])

#Compute the average adjusted profit% per day
m2['APPD'] = m2.apply(lambda row: row['AdjustedProfit%'] / row.TimeInDays, axis = 1)
m2['TradesPerDay'] = np.where(m2['TimeInDays'] >= 1, m2['NoOfTrades'] / m2['TimeInDays'], 0)
m2['TradesPerDay'] = m2['TradesPerDay'].apply(lambda x: f'{x:.1f}')
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var,ratio,RatioNegative,AdjustedProfit%,APPD,TradesPerDay
0,BINANCE:BNBUSDT.P,Strategy v1,1m,3.0,17.0576,0.0,0.0000,218.79,211.63,218.79,...,17.0576,12.371528,211.63,218.79,0.0%,100.0,0%,170.576,13.787788,0.2
1,BINANCE:BNBUSDT.P,Strategy v10,1m,2.0,4.4184,3.0,-3.0624,214.47,211.65,214.47,...,1.3560,8.086111,211.65,214.47,0.0%,1.4,60%,13.560,1.676950,0.6
2,BINANCE:BNBUSDT.P,Strategy v15,1m,5.0,27.5060,2.0,-5.6616,216.27,211.75,216.27,...,21.8444,13.589583,211.75,216.27,0.0%,4.9,29%,218.444,16.074371,0.5
3,BINANCE:BNBUSDT.P,Strategy v16,1m,1.0,0.5292,0.0,0.0000,213.46,213.46,213.46,...,0.5292,5.065972,213.46,213.46,0.0%,100.0,0%,5.292,1.044617,0.2
4,BINANCE:BNBUSDT.P,Strategy v19,1m,26.0,29.0092,36.0,-55.3288,218.59,204.60,218.59,...,-26.3196,17.374306,204.60,218.59,0.1%,0.5,58%,-263.196,-15.148577,3.6
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,113.0,105.7996,58.0,-71.3464,218.99,204.97,218.99,...,34.4532,16.847222,204.97,218.99,0.1%,1.5,34%,344.532,20.450374,10.2
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,16.0,35.2872,12.0,-29.3196,218.52,202.87,218.52,...,5.9676,13.674306,202.87,218.52,0.1%,1.2,43%,59.676,4.364097,2.0
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,1139.0,235.9688,1600.0,-175.7600,221.11,203.79,221.11,...,60.2088,17.629167,203.79,221.11,0.1%,1.3,58%,602.088,34.152947,155.4
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,138.0,72.8496,290.0,-76.1620,220.19,204.99,220.19,...,-3.3124,17.592361,204.99,220.19,0.1%,1.0,68%,-33.124,-1.882863,24.3
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,637.0,170.1804,1136.0,-144.0188,221.08,203.98,221.08,...,26.1616,17.614583,203.98,221.08,0.1%,1.2,64%,261.616,14.852239,100.7


In [19]:
################################################################################################################################
#Reorder the columns
cols2 = ['ticker', 'strategy', 'timeframe', 'entry_time', 'exit_time', 'TimeInDays', 
        'AvgTimeInMarketHours', 'StdTimeInMarket', 'NoOfTrades', 'RatioNegative', 'TotalProfit%', 'AvgProfitPerTrade%', 
        'ProfitPositive%', 'ProfitNegative%', 'ratio', 'MinCumulative%', 'MaxCumulative%', 'runup%', 'drawdown%', 
        'MinPrice', 'MaxPrice', 'var', 'TradesPerDay', 'AdjustedProfit%', 'APPD']

################################################################################################################################
#slice the df
df2 = m2[cols2]

In [20]:
################################################################################################################################
#sort by ratio
df2.sort_values(by=['ratio'], ascending=[False])

################################################################################################################################
#Save the analysis daily
os.chdir(r'D:\TD\FMF\Strategies\Results')
df2.to_csv('AllResults_OverallStrategy ' + today + '.csv', sep=',', index=False, header=True)

In [21]:
df2 #have a final look before starting to look at it in Excel

,ticker,strategy,timeframe,entry_time,exit_time,TimeInDays,AvgTimeInMarketHours,StdTimeInMarket,NoOfTrades,RatioNegative,...,MinCumulative%,MaxCumulative%,runup%,drawdown%,MinPrice,MaxPrice,var,TradesPerDay,AdjustedProfit%,APPD
0,BINANCE:BNBUSDT.P,Strategy v1,1m,2023-09-26 01:57:00,2023-10-08 10:52:00,12.371528,3.738704,1.172913,3,0%,...,1.77%,8.13%,3.1,2.9,211.63,218.79,0.0%,0.2,170.576,13.787788
1,BINANCE:BNBUSDT.P,Strategy v10,1m,2023-09-26 06:00:00,2023-10-04 08:04:00,8.086111,0.144444,0.086055,5,60%,...,-0.53%,1.97%,1.4,0.4,211.65,214.47,0.0%,0.6,13.560,1.676950
2,BINANCE:BNBUSDT.P,Strategy v15,1m,2023-09-26 01:54:00,2023-10-09 16:03:00,13.589583,0.985040,0.515875,7,29%,...,-2.85%,8.24%,2.3,1.0,211.75,216.27,0.0%,0.5,218.444,16.074371
3,BINANCE:BNBUSDT.P,Strategy v16,1m,2023-10-01 17:56:00,2023-10-06 19:31:00,5.065972,2.026389,NaN,1,0%,...,0.53%,0.53%,3.9,1.5,213.46,213.46,0.0%,0.2,5.292,1.044617
4,BINANCE:BNBUSDT.P,Strategy v19,1m,2023-09-25 01:45:00,2023-10-12 10:44:00,17.374306,0.123217,0.126077,62,58%,...,-0.30%,9.77%,0.5,0.9,204.60,218.59,0.1%,3.6,-263.196,-15.148577
5,BINANCE:BNBUSDT.P,Strategy v2.1,1m,2023-09-25 06:33:00,2023-10-12 02:53:00,16.847222,0.187326,0.173116,171,34%,...,-0.14%,9.92%,0.5,0.9,204.97,218.99,0.1%,10.2,344.532,20.450374
6,BINANCE:BNBUSDT.P,Strategy v9.1,1m,2023-09-27 06:25:00,2023-10-10 22:36:00,13.674306,0.295397,0.391121,28,43%,...,-0.62%,9.54%,0.9,1.3,202.87,218.52,0.1%,2.0,59.676,4.364097
7,BINANCE:BNBUSDT.P,[CP] ChannelBreakOutStrategy,1m,2023-09-25 00:00:00,2023-10-12 15:06:00,17.629167,0.002575,0.001906,2739,58%,...,-0.09%,9.98%,0.1,0.0,203.79,221.11,0.1%,155.4,602.088,34.152947
8,BINANCE:BNBUSDT.P,[CP] Keltner Channels Strategy,1m,2023-09-25 00:22:00,2023-10-12 14:35:00,17.592361,0.016441,0.013927,428,68%,...,-0.00%,2.66%,0.3,0.1,204.99,220.19,0.1%,24.3,-33.124,-1.882863
9,BINANCE:BNBUSDT.P,[CP] Momentum Strategy,1m,2023-09-25 00:20:00,2023-10-12 15:05:00,17.614583,0.003974,0.003397,1773,64%,...,-0.17%,9.93%,0.1,0.1,203.98,221.08,0.1%,100.7,261.616,14.852239
